In [4]:
pip install kafka-python pandas pyarrow

  Using cached pyarrow-17.0.0-cp310-cp310-win_amd64.whl.metadata (3.4 kB)
Using cached pyarrow-17.0.0-cp310-cp310-win_amd64.whl (25.1 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
from kafka import KafkaConsumer
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime

consumer = KafkaConsumer(
    'bitcoin-price-topic',  
    bootstrap_servers=['localhost:9092'],  
    auto_offset_reset='earliest',  
    enable_auto_commit=True,
    group_id='price-changes-group',  
    value_deserializer=lambda x: eval(x.decode('utf-8'))  
)

telegram_token = '6553143527:AAER2zcqewpgWYvXwlT-NwGLvarviLK3E-0'  
telegram_chat_id = '5736560518' 

def send_telegram_alert(message):
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    params = {
        'chat_id': telegram_chat_id,
        'text': message
    }
    requests.get(url, params=params)


price_threshold = 58000  


data_list = []


for message in consumer:
    
    message_value = message.value
    price = float(message_value['price'].replace('$', '').replace(',', ''))  

    data_list.append(message_value)

    if price < price_threshold:
        alert_message = f"⚠️ Giá Bitcoin đã giảm xuống dưới ${price_threshold}: Giá hiện tại là ${price}"
        send_telegram_alert(alert_message)
    
    if len(data_list) >= 5:
        df = pd.DataFrame(data_list)
        
        parquet_file = f"data-coin/price_changes_{datetime.now().strftime('%Y%m%d_%H%M%S')}.parquet"

        table = pa.Table.from_pandas(df)
        pq.write_table(table, parquet_file)

        print(f"Đã lưu dữ liệu vào {parquet_file}")

        data_list.clear()


Đã lưu dữ liệu vào data-coin/price_changes_20240913_151702.parquet


KeyboardInterrupt: 